In [1]:
import numpy as np

# define Pauli matrices
I = np.array([[1, 0], [0, 1]], dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

# define the Hadamard gate
H = (1/np.sqrt(2)) * np.array([[1, 1], [1, -1]], dtype=complex)

def stabilizer(type, list, num_qubits):
    """
    Create a stabilizer operator for the Steane code.
    
    Parameters:
    type (str): 'X' or 'Z' to indicate the type of stabilizer.
    list (list): List of qubit indices that the stabilizer acts on.
    num_qubits (int): Total number of qubits in the code.
    
    Returns:
    np.ndarray: The stabilizer operator as a tensor product of Pauli matrices.
    """
    if type == 'X':
        pauli = X
    elif type == 'Z':
        pauli = Z
    else:
        raise ValueError("Type must be 'X' or 'Z'")
    
    # Start with identity for all qubits
    stabilizer_op = I
    for i in range(num_qubits):
        if i in list:
            stabilizer_op = np.kron(stabilizer_op, pauli)
        else:
            stabilizer_op = np.kron(stabilizer_op, I)
    
    return stabilizer_op

# define the stabilizers for the Steane code, which is a 7-qubit code
# The qubits are labelled as 0 to 6
# The stabilizers are defined as follows:
# S_(X,1) = X(0,1,2,3), S_(X,2) = X(1,2,4,5), S_(X,3) = X(2,3,5,6)
# S_(Z,1) = Z(0,1,2,3), S_(Z,2) = Z(1,2,4,5), S_(Z,3) = Z(2,3,5,6)
stabilizers = {
    'S_X1': stabilizer('X', [0, 1, 2, 3], 7),
    'S_X2': stabilizer('X', [1, 2, 4, 5], 7),
    'S_X3': stabilizer('X', [2, 3, 5, 6], 7),
    'S_Z1': stabilizer('Z', [0, 1, 2, 3], 7),
    'S_Z2': stabilizer('Z', [1, 2, 4, 5], 7),
    'S_Z3': stabilizer('Z', [2, 3, 5, 6], 7)
}

def projector(stabilizer_gen, num_qubits=7):
    """
    Create a projector onto the code space defined by a list of stabilizer generators.
    The projector is defined by the product of all (I + S) for each generator S.

    Parameters:
    stabilizer_gen (list): List of stabilizer generators.

    Returns:
    np.ndarray: The projector operator.
    """
    identity = stabilizer('X',[], num_qubits)  # Identity operator for the projector
    projector_op = identity
    for gen in stabilizer_gen:
        projector_op = np.matmul(projector_op, (identity + gen))
    projector_op = projector_op / np.trace(projector_op)  # Normalize the projector
    return projector_op

In [2]:
# Define the 7-qubit product state |0000000>, which is tensor product of (1,0) 

state = np.array([1, 0], dtype=complex)  # |0>
for _ in range(6):
    state = np.kron(state, np.array([1, 0], dtype=complex))  # Tensor product

# Calculate the projector onto the code space
projector_op = projector(list(stabilizers.values()))
# Apply the projector to the state
projected_state = np.matmul(projector_op, state)
# Normalize the projected state
projected_state /= np.linalg.norm(projected_state)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 128 is different from 256)

In [3]:
import numpy as np

# Define the Hadamard gate H
H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)

# Define the phase gate S
S = np.array([[1, 0], [0, 1j]])

# Compute HSH = H @ S @ H
HSH = H @ S @ H

# Calculate eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(HSH)

# Print results
print("HSH matrix:")
print(HSH)
print("\nEigenvalues:")
print(eigenvalues)
print("\nEigenvectors (as columns):")
print(eigenvectors)

HSH matrix:
[[0.5+0.5j 0.5-0.5j]
 [0.5-0.5j 0.5+0.5j]]

Eigenvalues:
[ 1.00000000e+00-5.55111512e-17j -8.32667268e-17+1.00000000e+00j]

Eigenvectors (as columns):
[[ 0.70710678-1.11022302e-16j  0.70710678+0.00000000e+00j]
 [ 0.70710678+0.00000000e+00j -0.70710678-4.16333634e-17j]]
